In [1]:
%load_ext autoreload
%autoreload 2

import sys         
import numpy as np
import torch

sys.path.append('./../src/') 

In [2]:
from data.sleep_edf import load_sleep_edf_split, create_sleep_edf_dataset
from data.ptbxl import load_ptbxl_split, create_ptbxl_dataset
from data.representations import Representation, PeriodicRepresentation

WAVEFORMS_REPRESENTATIONS = [
    Representation.WHOLE_SIGNAL_WAVEFORMS,
    Representation.WINDOWS_WAVEFORMS,
    PeriodicRepresentation.BEATS_WAVEFORMS,
    PeriodicRepresentation.AGG_BEAT_WAVEFORMS
]

FEATURES_REPRESENTATION = [
    Representation.WHOLE_SIGNAL_FEATURS,
    Representation.WINDOWS_FEATURES,
    PeriodicRepresentation.BEATS_FEATURES,
    PeriodicRepresentation.AGG_BEAT_FEATURES

]

In [ ]:
create_sleep_edf_dataset(representation_types=[Representation.WINDOWS_FEATURES, Representation.WINDOWS_WAVEFORMS])

Creating representations for all splits:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting representations:   0%|          | 0/135740 [00:00<?, ?it/s]

In [21]:
create_ptbxl_dataset(representation_types=[Representation.WHOLE_SIGNAL_WAVEFORMS, Representation.WINDOWS_WAVEFORMS])

Creating representations for all splits:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting representations:   0%|          | 0/12978 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/shate/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_10269/1231420374.py", line 1, in <cell line: 1>
    create_ptbxl_dataset(representation_types=[Representation.WHOLE_SIGNAL_WAVEFORMS, Representation.WINDOWS_WAVEFORMS])
  File "/home/shate/Desktop/praca_magisterska/medical_signal_representation/notebooks/./../src/data/ptbxl.py", line 163, in create_ptbxl_dataset
    create_dataset(
  File "/home/shate/Desktop/praca_magisterska/medical_signal_representation/notebooks/./../src/data/representations.py", line 64, in create_dataset
    representation_path = representations_path / name
  File "/home/shate/Desktop/praca_magisterska/medical_signal_representation/notebooks/./../src/data/representations.py", line 64, in <listcomp>
    representation_path = representations_path / name
  File "/home/shate/Desktop/praca_magiste

# **TODO**
* Create every dataset representation for PTB-XL and Sleep-EDF (and check if all works fine)
* Get work done for MIMIC dataset
* Think about adding EMG and EOG for Sleep-EDF 